In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. tensorflow v2.10에서 v1사용하기

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() # tensorflow v2 비활성화하고 v1활성화
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


## Tensorflow

- 데이터 흐름 그래프(tensor 객체의 흐름)을 사용하는 수치 계산 라이브러리
- 그래프는 node(연산)와 edge로 구성
- sess = tf.Session()을 이용해서 실행환경 
- sess.run()을 통해서 실행 결과를 확인

In [6]:
# 1단계. tensor(상수 노드 하나) 정의
node1 = tf.constant('Hello, Tensorflow')

# 2단계. 세션(연산을 실행하는 환경) 생성
sess = tf.Session()
print(node1)

# 3단계. 실제 실행 및 출력
print(sess.run(node1)) 
print(sess.run(node1).decode()) # 바이너리가 같이 출력되는게 싫을때

Tensor("Const_2:0", shape=(), dtype=string)
b'Hello, Tensorflow'
Hello, Tensorflow


In [7]:
# 간단한 연산 tensor그래프

# 1단계. 그래프 전의
node1 = tf.constant(10, dtype=tf.float16)
node2 = tf.constant(20, dtype=tf.float16)
node3 = tf.add(node1,node2)

# 2단계. 세션 생성
sess = tf.Session()

# 3단계. 세션 실행 & 결과
print(sess.run([node1, node2, node3]))

[10.0, 20.0, 30.0]


In [8]:
# 타입 변경

import numpy as np
node1 = tf.constant(np.array([1,2,3]), dtype=tf.int16)
node2 = tf.cast(node1, dtype=tf.float32)
sess = tf.Session()
print(sess.run([node1, node2]))

[array([1, 2, 3], dtype=int16), array([1., 2., 3.], dtype=float32)]


In [10]:
# 평균값 계산 : tf.reduce_mean()

data = np.array([1.,2.,3.,4.])
m = tf.reduce_mean(data)
sess = tf.Session()
print(sess.run(m)) # data에 있는게 정수면 정수로 출력해서 2.5가 나와야하는데 2가 나옴.

2.5


In [12]:
# tf.random_normal([size]) : 평균이 0이고 표준편차가 1인 난수 size개 발생
w = tf.random_normal([2])
sess = tf.Session()
sess.run(w)

array([-1.5833478,  0.7646064], dtype=float32)

In [13]:
# 변수 노드. variable에 넣어주는 이유는 random_normal 안에 값을 우리가 조정할 수 없기 때문에

w = tf.Variable(tf.random_normal([1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 변수 초기화
sess.run(w)

array([-0.12919138], dtype=float32)

# 2. tensorflow v1을 이용한 회귀분석 구현

## 2.1 독립변수 x가 1개, 타겟변수(종속변수) y가 1개

In [16]:
# tensor 그래프 정의

# 1. 데이터 셋 확보
x = np.array([1,2,3])
y = np.array([2,3,4]) # y = 1*x + 1

# 2. weight와 bias
W = tf.Variable(tf.random_normal([1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# 3. Hat, Hypothesis : 결과는 numpy 배열
H = W * x + b

# 4. cost function(손실함수 : mse)
cost = tf.reduce_mean(tf.square(H - y)) # 제곱함수

'''
학습목적 : cost가 최소가 되는 w, b를 찾는 것
cost 함수는 2차함수 이므로 곡선 그래ㅍ, 곡선 위 미분값이 0이 되는 방향(경사하강법 GradientDescent)
'''

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01) # learning_rate이 크면 학습을 성큼성큼함
train = optimizer.minimize(cost)

# 세션 생성
sess = tf.Session()

# W,b 초기화
sess.run(tf.global_variables_initializer())

# 6000번 학습 (v2에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b])
    if step%300==0:
        print(f'{step}번째 cost:{cost_val}, W:{W_val}, b:{b_val}')

0번째 cost:60.98166275024414, W:[-1.3477035], b:[-0.97893333]
300번째 cost:0.021376444026827812, W:[1.1694019], b:[0.6149095]
600번째 cost:0.005043968092650175, W:[1.0822883], b:[0.8129398]
900번째 cost:0.001190171460621059, W:[1.0399717], b:[0.90913415]
1200번째 cost:0.0002808296412695199, W:[1.0194166], b:[0.9558615]
1500번째 cost:6.626448885072023e-05, W:[1.0094316], b:[0.97855955]
1800번째 cost:1.563679688842967e-05, W:[1.0045817], b:[0.9895847]
2100번째 cost:3.6900394206895726e-06, W:[1.0022258], b:[0.9949405]
2400번째 cost:8.713172974239569e-07, W:[1.0010817], b:[0.9975415]
2700번째 cost:2.0606141504231346e-07, W:[1.000526], b:[0.9988043]
3000번째 cost:4.8923777029585835e-08, W:[1.0002565], b:[0.9994176]
3300번째 cost:1.1623683349171188e-08, W:[1.0001249], b:[0.99971604]
3600번째 cost:2.7853275241795927e-09, W:[1.0000614], b:[0.99986106]
3900번째 cost:6.668017249467084e-10, W:[1.0000299], b:[0.9999319]
4200번째 cost:1.7485390912952425e-10, W:[1.0000155], b:[0.9999652]
4500번째 cost:5.328478283606053e-11, W:[1.0

In [18]:
# 최종적으로 나온 회귀식 H = W*x + b

W_, b_ = sess.run([W,b])

In [19]:
def predict(x):
    return W_[0] * x + b_[0]

In [20]:
predict(5)

6.000024616718292

## 2.2 predict을 위한 placeholder이용

- placeholder : 외부에서 데이터를 입력받을 수 있도록 하는 노드

In [ ]:
# 이런건 데이터가 고정임
x = np.array([1,2,3])
H = 1*x + 1

In [22]:
x = tf.placeholder(dtype=tf.float32) # x의 값이 float32로 들어올 예정이야. 아직은 몰라
H = 1*x +1
sess = tf.Session()
sess.run([x, H], feed_dict={x : np.array([1,2,3])}) # feed_dict 먹이를 물어온다

[array([1., 2., 3.], dtype=float32), array([2., 3., 4.], dtype=float32)]

In [23]:
# tensor 그래프 정의

# 1. 데이터 셋 확보
x_data = np.array([1,2,3])
y_data = np.array([2,3,4]) # y = 1*x + 1

# placeholder 설정 (x,y)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)

# 2. weight와 bias
W = tf.Variable(tf.random_normal([1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# 3. Hat, Hypothesis : 결과는 numpy 배열
H = W*x + b

# 4. cost function(손실함수 : mse)
cost = tf.reduce_mean(tf.square(H - y)) # 제곱함수

'''
학습목적 : cost가 최소가 되는 w, b를 찾는 것
cost 함수는 2차함수 이므로 곡선 그래ㅍ, 곡선 위 미분값이 0이 되는 방향(경사하강법 GradientDescent)
'''

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01) # learning_rate이 크면 학습을 성큼성큼함
train = optimizer.minimize(cost)

# 세션 생성
sess = tf.Session()

# W,b 초기화
sess.run(tf.global_variables_initializer())

# 6000번 학습 (v2에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b], feed_dict={x:x_data, y:y_data})
    if step%300==0:
        print(f'{step}번째 cost:{cost_val}, W:{W_val}, b:{b_val}')

0번째 cost:3.812636137008667, W:[0.6552528], b:[-0.02816962]
300번째 cost:0.01834465004503727, W:[1.1569299], b:[0.6432615]
600번째 cost:0.004328586161136627, W:[1.0762296], b:[0.8267122]
900번째 cost:0.0010213690111413598, W:[1.0370289], b:[0.91582453]
1200번째 cost:0.00024100382870528847, W:[1.0179873], b:[0.95911074]
1500번째 cost:5.6866370869101956e-05, W:[1.0087372], b:[0.9801381]
1800번째 cost:1.3418551134236623e-05, W:[1.0042443], b:[0.99035186]
2100번째 cost:3.1664801554143196e-06, W:[1.0020617], b:[0.99531335]
2400번째 cost:7.478130896743096e-07, W:[1.0010021], b:[0.9977225]
2700번째 cost:1.7690588549612585e-07, W:[1.0004876], b:[0.99889225]
3000번째 cost:4.1997832767037835e-08, W:[1.0002375], b:[0.9994603]
3300번째 cost:9.991576455092854e-09, W:[1.0001159], b:[0.999737]
3600번째 cost:2.4127568831744384e-09, W:[1.0000573], b:[0.99987054]
3900번째 cost:5.62541402349126e-10, W:[1.0000273], b:[0.9999377]
4200번째 cost:1.5669836261889003e-10, W:[1.0000147], b:[0.9999671]
4500번째 cost:5.195962063386794e-11, W:[1

In [24]:
# 예측하기

sess.run(H, feed_dict={x:5})

array([6.0000243], dtype=float32)

## 2.3 scale이 다른 데이터들의 회귀분석 구현(scale 조정X)

In [28]:
# 데이터셋 확보
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])

# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)

# weight & bias
W = tf.Variable(tf.random_normal([1]))
b = tf.Variable(tf.random_normal([1]))

# Hypothesis
H = W*x + b

# cost(손실함수)
cost = tf.reduce_mean(tf.square(H-y))

# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)

# Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(1,6001):
    _, cost_val = sess.run([train, cost], feed_dict={x:x_data, y:y_data})
    if step%300==0:
        print(f'{step}/6000번째 cost:{cost_val}')

300/6000번째 cost:79.84456634521484
600/6000번째 cost:79.63267517089844
900/6000번째 cost:79.4845199584961
1200/6000번째 cost:79.38082885742188
1500/6000번째 cost:79.30831909179688
1800/6000번째 cost:79.25756072998047
2100/6000번째 cost:79.22209167480469
2400/6000번째 cost:79.197265625
2700/6000번째 cost:79.17989349365234
3000/6000번째 cost:79.16771697998047
3300/6000번째 cost:79.15924072265625
3600/6000번째 cost:79.15330505371094
3900/6000번째 cost:79.14913940429688
4200/6000번째 cost:79.1462173461914
4500/6000번째 cost:79.14420318603516
4800/6000번째 cost:79.14278411865234
5100/6000번째 cost:79.14178466796875
5400/6000번째 cost:79.1410903930664
5700/6000번째 cost:79.14060974121094
6000/6000번째 cost:79.14024353027344


## 2.4 scale이 다른 데이터들의 회귀분석 구현(조정O)

### sclae 조정방법 : 모든 데이터를 일정범위내로 조정

- normalization(정규화) : 모든 데이터를 0~1 사이로 조정
    - 아래의 식보다 라이브러리 추천(sklearn.preprocessing.MinMaxScaler)
```    
                         X - Xmin
    normalization = ───────
                        Xmax - Xmin
```

- standarization(표준화) : 데이터의 평균을 0, 표준편차를 1로 조정
    - 아래의 식보다 라이브러리 추천(sklearn.preprocessing.StandardScaler)

```    
                        X - Xmean
    standarization = ────────
                      Xstd(표준편차)
```

In [32]:
## 라이브러리 쓰지않고 정규화 ##

# 데이터셋 확보
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])

# 정규화
norm_scaled_x_data = (x_data - x_data.min()) / (x_data.max() - x_data.min())
norm_scaled_y_data = (y_data - y_data.min()) / (y_data.max() - y_data.min())

array([0.        , 0.11111111, 0.7       , 0.83333333, 1.        ])

In [3]:
## 라이브러리 사용하여 정규화 ##

# 데이터셋 확보
x_data = np.array([1,2,5,8,10]).reshape(-1,1)
y_data = np.array([5,15,68,80,95]).reshape(-1,1)

from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler_x = MinMaxScaler() # 독립변수 x를 정규화시킬 객체
norm_scaled_x_data = scaler_x.fit_transform(x_data)
scaler_y = MinMaxScaler() # 종속(타겟)변수 y를 정규화시킬 객체
norm_scaled_y_data = scaler_y.fit_transform(y_data)

In [38]:
## 라이브러리 쓰지않고 표준화 ##

# 데이터셋 확보
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])

# 표준화
stan_scaled_x_data = (x_data - x_data.mean()) / x_data.std()
stan_scaled_y_data = (y_data - y_data.mean()) / y_data.std()
np.column_stack([x_data, norm_scaled_x_data, stan_scaled_x_data])

array([[ 1.        ,  0.        , -1.22474487],
       [ 2.        ,  0.11111111, -0.93313895],
       [ 5.        ,  0.44444444, -0.05832118],
       [ 8.        ,  0.77777778,  0.81649658],
       [10.        ,  1.        ,  1.39970842]])

In [39]:
np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [2]:
## 라이브러리 사용하여 표준화 ##

# 데이터셋 확보
x_data = np.array([1,2,5,8,10]).reshape(-1,1)
y_data = np.array([5,15,68,80,95]).reshape(-1,1)

from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
stan_scaled_x_data = scaler_x.fit_transform(x_data)
scaler_y = StandardScaler()
stan_scaled_y_data = scaler_y.fit_transform(y_data)

In [41]:
np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [42]:
## 스케일 조정된 데이터를 다시 복구 : inverse_transform() 이용

scaler_y.inverse_transform(stan_scaled_y_data)

array([[ 5.],
       [15.],
       [68.],
       [80.],
       [95.]])

In [4]:
# 데이터셋 확보
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])

# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)

# weight & bias
W = tf.Variable(tf.random_normal([1]))
b = tf.Variable(tf.random_normal([1]))

# Hypothesis
H = W*x + b

# cost(손실함수)
cost = tf.reduce_mean(tf.square(H-y))

# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)

# Session & 변수 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(1,6001):
    _, cost_val = sess.run([train, cost], feed_dict={x:norm_scaled_x_data, y:norm_scaled_y_data})
    if step%300==0:
        print(f'{step}/6000번째 cost:{cost_val}')

300/6000번째 cost:0.6020394563674927
600/6000번째 cost:0.20580962300300598
900/6000번째 cost:0.10889718681573868
1200/6000번째 cost:0.08001042902469635
1500/6000번째 cost:0.06727872043848038
1800/6000번째 cost:0.05897826701402664
2100/6000번째 cost:0.05238027125597
2400/6000번째 cost:0.046781618148088455
2700/6000번째 cost:0.04194432497024536
3000/6000번째 cost:0.03774500638246536
3300/6000번째 cost:0.034095071256160736
3600/6000번째 cost:0.030921638011932373
3900/6000번째 cost:0.02816227450966835
4200/6000번째 cost:0.025762910023331642
4500/6000번째 cost:0.023676570504903793
4800/6000번째 cost:0.02186240255832672
5100/6000번째 cost:0.02028490975499153
5400/6000번째 cost:0.01891321875154972
5700/6000번째 cost:0.01772046647965908
6000/6000번째 cost:0.016683321446180344
